In [202]:
sealed trait Stream[+A]{
    def headOption: Option[A] = this match {
        case Empty => None
        case Cons(h, t) => Some(h())
    }
    
    // 5.1
    def toList: List[A] = this match{
        case Empty => Nil
        case Cons(h,t) => h() :: t().toList
    }
    
    //5.2
    def take(n:Int):List[A] = this match{
        case Cons(h,t) if (n > 0) => h()::t().take(n-1)
        case _  => Nil
    }
    
    def drop(n:Int): List[A] = this match {
        case Cons(h,t) if n > 0 => t().drop(n-1)
        case _ => this.toList
    }

    
    //5.3
    def takeWhile(p: A => Boolean): Stream[A] = this match {
        case Empty => Empty
        case Cons(h,t) => if (p(h())) Cons(h,()=>t().takeWhile(p)) else Empty
    }
    
    def foldRight[B](z: => B)(f: (A, => B) => B) : B = this match {
        case Cons(h,t) => f(h(), t().foldRight(z)(f))
        case _ => z
    }
    
    def exists(f: A => Boolean):Boolean = {
        this.foldRight(false)((x,xs) => f(x) || xs)
    }
    
    //5.4
    def forAll(p:A => Boolean): Boolean = {
        this.foldRight(true)((a,b) => p(a) && b)
    }
    
    //5.5
    def takeWhile2(f:A => Boolean):Stream[A] = {
        this.foldRight(Empty:Stream[A])(((x,xs) => if (f(x)) Cons(() => x,() =>xs) else Empty ))
    }
    
    //5.6
    def headOption2():Option[A] = {
        this.foldRight(None:Option[A])((x,xs) => Some(x))
    }
    
    //5.7
    def map2[B](f:A => B):Stream[B] = {
        this.foldRight(Empty:Stream[B])((x,xs) => Cons(() => f(x), () => xs))
    }
    
    def filter2(f:A => Boolean):Stream[A] = {
        this.foldRight(Empty:Stream[A])((x,xs) => if (f(x)) Cons(() => x, () => xs) else xs)
    }
    
    def append[B >: A](as: => Stream[B]): Stream[B] = {
        this.foldRight(as)((x,xs) => Cons(() => x,() => xs))
    } 
    def flatMap[B](f:A => Stream[B]): Stream[B] = {
        this.foldRight(Empty:Stream[B])((x,xs) => f(x).append(xs))
    }
    
    def find(f: A => Boolean):Option[A] = {
        filter2(f).headOption2()
    }
    
}
case object Empty extends Stream[Nothing]
case class Cons[+A](h: () => A, t: () => Stream[A]) extends Stream[A]

object Stream{
    def cons[A](hd: => A, tl: => Stream[A]): Stream[A] = {
    lazy val head = hd
    lazy val tail = tl
    Cons(() => head, () => tail)
  }

    def empty[A]: Stream[A] = Empty

    def apply[A](as: A*): Stream[A] = {
        if (as.isEmpty) empty
        else cons(as.head, apply(as.tail: _*))
  }
    
    
}



defined trait Stream
defined object Empty
defined class Cons
defined object Stream


In [203]:
val ss = Stream.apply(1,2,3,4,5)

ss: Stream[Int] = Cons(Stream$$$Lambda$2766/0x0000000801a06840@68716a34,Stream$$$Lambda$2767/0x0000000801a06c40@164c5890)


In [204]:
ss.take(3)

res93: List[Int] = List(1, 2, 3)


In [205]:
ss.drop(3)

res94: List[Int] = List(4, 5)


In [206]:
ss.takeWhile(a => a%2 == 1).toList

res95: List[Int] = List(1)


In [207]:
ss.forAll(_>3)

res96: Boolean = false


In [208]:
ss.takeWhile2(a => a%2 == 1).toList

res97: List[Int] = List(1)


In [209]:
ss.map2(_*2).toList

res98: List[Int] = List(2, 4, 6, 8, 10)


In [210]:
ss.filter2(a => a%2 == 0).toList

res99: List[Int] = List(2, 4)


In [211]:
ss.append(Stream.apply(66,55,44)).toList

res100: List[Int] = List(1, 2, 3, 4, 5, 66, 55, 44)


In [212]:
ss.flatMap(x=>Stream(x,x)).toList

res101: List[Int] = List(1, 1, 2, 2, 3, 3, 4, 4, 5, 5)


In [213]:
ss.find(a => a%2 == 1).getOrElse(None)

res102: Any = 1


## 무한 스트림

In [214]:
val ones: Stream[Int] = Stream.cons(1,ones)

ones: Stream[Int] = Cons(Stream$$$Lambda$2766/0x0000000801a06840@19ad3902,Stream$$$Lambda$2767/0x0000000801a06c40@6f9db82c)


In [215]:
ones.take(5)

res103: List[Int] = List(1, 1, 1, 1, 1)


In [218]:
ones.map2(_ + 1).exists(_ % 2 == 0)

res106: Boolean = true


In [221]:
ones.takeWhile(_ == 1)

res109: Stream[Int] = Cons(Stream$$$Lambda$2766/0x0000000801a06840@19ad3902,Stream$$Lambda$2769/0x0000000801a06440@742d725d)


In [223]:
ones.forAll(_ != 1)

res111: Boolean = false


In [224]:
def constant[A](a:A): Stream[A] = {
    Stream.cons(a,constant(a))
}

constant: [A](a: A)Stream[A]


In [225]:
def from(n:Int): Stream[Int] = {
    Stream.cons(n,from(n+1))
}

from: (n: Int)Stream[Int]


In [229]:
def fibs():Stream[Int] = {
    def go(a:Int,b:Int):Stream[Int] = {
        Stream.cons(a,go(b,a+b))
    }
    go(0,1)
   
}

fibs: ()Stream[Int]


In [233]:
fibs().take(7)

res114: List[Int] = List(0, 1, 1, 2, 3, 5, 8)


In [269]:
def unfold[A,S](z:S)(f: S => Option[(A,S)]):Stream[A] = f(z) match {
    case None => Empty
    case Some((a,s)) => Stream.cons(a,unfold(s)(f))
}

unfold: [A, S](z: S)(f: S => Option[(A, S)])Stream[A]


In [270]:
val ones2:Stream[Int] = unfold(1)(a =>Some((a,a)))

ones2: Stream[Int] = Cons(Stream$$$Lambda$2766/0x0000000801a06840@557353b5,Stream$$$Lambda$2767/0x0000000801a06c40@5d4e6044)


In [271]:
ones2.take(5)

res121: List[Int] = List(1, 1, 1, 1, 1)


In [272]:
def constant2[A](a:A):Stream[A] = {
    unfold(a)(s =>Some((s,s)))
}

constant2: [A](a: A)Stream[A]


In [273]:
constant(5).take(5)

res122: List[Int] = List(5, 5, 5, 5, 5)


In [276]:
def from2(a:Int):Stream[Int] = {
    unfold(a)(s => Some((s,s+1)))
}

from2: (a: Int)Stream[Int]


In [277]:
from2(1).take(10)

res124: List[Int] = List(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)


In [283]:
def fibs2():Stream[Int] = {
    def go(a:Int,b:Int):Stream[Int] = {
        unfold((a,b))((s:(Int,Int)) => Some(s._1,(s._2,s._1+s._2)))
    }
    go(0,1)
   
}

fibs2: ()Stream[Int]


In [286]:
fibs2().take(10)

res127: List[Int] = List(0, 1, 1, 2, 3, 5, 8, 13, 21, 34)
